In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
import random
from sklearn.metrics import r2_score

<code>Gradient Descent For 2 Columns</code>

In [2]:

df = pd.read_csv('./Data/simple_linear_regression_data.csv')
X = df.iloc[:, 1].values.reshape(-1, 1)
y = df.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
class GDRegressor():
  def __init__(self,learning_score,epochs):
    self.m=0
    self.b=0
    self.lr=learning_score
    self.epochs=epochs
  def fit(self,X_train,y_train):
    for i in range(self.epochs):
      loss_slope_b = -2 * np.sum(y_train - self.m*X_train.ravel() - self.b)
      loss_slope_m = -2 * np.sum((y_train - self.m*X_train.ravel() - self.b)*X_train.ravel())
      self.b = self.b -(self.lr*loss_slope_b)
      self.m = self.m -(self.lr*loss_slope_m)
    print(self.m,self.b)
  def predict(self,X_test):
    return self.m*X_test + self.b
  
gd = GDRegressor(0.01,50)
gd.fit(X_train,y_train)

-6.691339362005465e+87 -8.19343792344914e+86


In [35]:
y_pred = gd.predict(X_test)
r2_score(y_test,y_pred)

0.2448591851218348

<code>Gradient Descent For n Columns or batch GD</code>
<pre>This GD update the parameter using entire data in each step
This is good for small data</pre>

In [6]:

X,y = load_diabetes(return_X_y=True)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [39]:
class GDRegressor():
  def __init__(self,learning_score,epochs):
    self.coef_ = None
    self.intercept_ = None
    self.lr = learning_score
    self.epochs = epochs

  def fit(self,X_train,y_train):
    self.intercept_ = 0
    self.coef_ = np.ones(X_train.shape[1])

    for i in range(self.epochs):
      y_hat = self.intercept_ + np.dot(X_train,self.coef_)
      intercept_der = -2 * np.mean(y_train - y_hat)
      self.intercept_ = self.intercept_ - (self.lr * intercept_der)

      coef_der = -2 * np.dot((y_train - y_hat),X_train)/X_train.shape[0]
      self.coef_ = self.coef_ - (self.lr * coef_der)

    print(self.coef_,self.intercept_)
  
  def predict(self,X_test):
    return np.dot(X_test,self.coef_) + self.intercept_
  


gd = GDRegressor(0.1,1000)
gd.fit(X_train,y_train)

[  62.27835432  -24.14017912  262.40285385  192.20751489   39.48809013
   10.26886323 -142.50597903  124.33312557  244.33510843  119.34350233] 151.94042847773682


In [40]:
y_pred=gd.predict(X_test)

In [ ]:
r2_score(y_test,y_pred)

0.3971698388048742

<code>Stochastic GD</code>
<pre>SGD update the parameter using single row in each step
This is good for large data
This is faster then BGS because it take less epochs to reach the answer</pre>

In [50]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [51]:
class SGDRegressor():
  def __init__(self,learning_rate=0.1,epochs=100):
    self.lr = learning_rate
    self.epochs = epochs
    self.coef_ = None
    self.intercept_ =None
  
  def fit(self,X_train,y_train):

    self.intercept_= 0
    self.coef_ = np.ones(X_train.shape[1])


    for i in range(self.epochs):
      for j in range(X_train.shape[0]):
        idx = random.randint(0,X_train.shape[0]-1)

        y_hat = np.dot(X_train[idx],self.coef_) + self.intercept_
        intercept_der = -2 * (y_train[idx] - y_hat)
        self.intercept_ = self.intercept_ - (self.lr * intercept_der)

        coef_der = -2 * np.dot((y_train[idx] - y_hat),X_train[idx])
        self.coef_ = self.coef_ - (self.lr * coef_der)
    print(self.coef_,self.intercept_)
  
  def predict(self,X_test):
    return np.dot(X_test,self.coef_) + self.intercept_

In [73]:
sgd = SGDRegressor(0.1,10)
sgd.fit(X_train,y_train)

[  33.5974029  -143.90626968  445.87387804  294.51197549  -45.89083794
 -116.55133006 -192.95520664  105.30980621  412.73236318   98.77398122] 151.02138082689302


In [76]:
y_pred = sgd.predict(X_test)
r2_score(y_test,y_pred)


0.45340073594010377

<code>Mini-Batch GD</code>
<pre>MBGD update the parameter using small small batch
ex: row = 1000 , batch_size = 10 , per epochs 100 row
This is good for large data
This is faster then BGS because it take less epochs to reach the answer</pre>

In [82]:
class MBGDRegressor():
  def __init__(self,batch_size,learning_rate=0.1,epochs=100):
    self.lr = learning_rate
    self.epochs = epochs
    self.coef_ = None
    self.intercept_ =None
    self.batch_size = batch_size
  
  def fit(self,X_train,y_train):

    self.intercept_= 0
    self.coef_ = np.ones(X_train.shape[1])


    for i in range(self.epochs):
      for j in range(int(X_train.shape[0]/self.batch_size)):
        idx = random.sample(range(X_train.shape[0]),self.batch_size)

        y_hat = np.dot(X_train[idx],self.coef_) + self.intercept_
        intercept_der = -2 * np.mean((y_train[idx] - y_hat))
        self.intercept_ = self.intercept_ - (self.lr * intercept_der)

        coef_der = -2 * np.dot((y_train[idx] - y_hat),X_train[idx])
        self.coef_ = self.coef_ - (self.lr * coef_der)
    print(self.coef_,self.intercept_)
  
  def predict(self,X_test):
    return np.dot(X_test,self.coef_) + self.intercept_

In [108]:
mrb = MBGDRegressor(batch_size=int(X_train.shape[0]/10),learning_rate=0.1,epochs=10)
mrb.fit(X_train,y_train)

[  40.61127011 -131.03298936  443.32755175  302.8758411   -11.66176732
  -94.22755413 -175.83314242  108.65025447  426.18234941  136.40990018] 146.7646415129892


In [109]:
y_pred = mrb.predict(X_test)
r2_score(y_test,y_pred)

0.44093834168519574